# Identifying safe loans with decision trees

The [LendingClub](https://www.lendingclub.com/) is a peer-to-peer leading company that directly connects borrowers and potential lenders/investors. In this notebook, you will build a classification model to predict whether or not a loan provided by LendingClub is likely to [default](https://en.wikipedia.org/wiki/Default_%28finance%29).

In this notebook you will use data from the LendingClub to predict whether a loan will be paid off in full or the loan will be [charged off](https://en.wikipedia.org/wiki/Charge-off) and possibly go into default. In this assignment you will:

* Use SFrames to do some feature engineering.
* Train a decision-tree on the LendingClub dataset.
* Predict whether a loan will default along with prediction probabilities (on a validation set).
* Train a complex tree model and compare it to simple tree model.

Let's get started!

## Fire up Turi Create

Make sure you have the latest version of Turi Create. If you don't find the decision tree module, then you would need to upgrade Turi Create using

```
   pip install turicreate --upgrade
```

In [1]:
import turicreate as tc

# Load LendingClub dataset

We will be using a dataset from the [LendingClub](https://www.lendingclub.com/). A parsed and cleaned form of the dataset is availiable [here](https://github.com/learnml/machine-learning-specialization-private). Make sure you **download the dataset** before running the following command.

In [2]:
loans = tc.SFrame('lending-club-data.sframe/')

In [3]:
loans.print_rows(num_columns=68)

+---------+-----------+-----------+-------------+-----------------+------------+
|    id   | member_id | loan_amnt | funded_amnt | funded_amnt_inv |    term    |
+---------+-----------+-----------+-------------+-----------------+------------+
| 1077501 |  1296599  |    5000   |     5000    |       4975      |  36 months |
| 1077430 |  1314167  |    2500   |     2500    |       2500      |  60 months |
| 1077175 |  1313524  |    2400   |     2400    |       2400      |  36 months |
| 1076863 |  1277178  |   10000   |    10000    |      10000      |  36 months |
| 1075269 |  1311441  |    5000   |     5000    |       5000      |  36 months |
| 1072053 |  1288686  |    3000   |     3000    |       3000      |  36 months |
| 1071795 |  1306957  |    5600   |     5600    |       5600      |  60 months |
| 1071570 |  1306721  |    5375   |     5375    |       5350      |  60 months |
| 1070078 |  1305201  |    6500   |     6500    |       6500      |  60 months |
| 1069908 |  1305008  |   12

## Exploring some features

Let's quickly explore what the dataset looks like. First, let's print out the column names to see what features we have in this dataset.

In [4]:
loans.column_names()

['id',
 'member_id',
 'loan_amnt',
 'funded_amnt',
 'funded_amnt_inv',
 'term',
 'int_rate',
 'installment',
 'grade',
 'sub_grade',
 'emp_title',
 'emp_length',
 'home_ownership',
 'annual_inc',
 'is_inc_v',
 'issue_d',
 'loan_status',
 'pymnt_plan',
 'url',
 'desc',
 'purpose',
 'title',
 'zip_code',
 'addr_state',
 'dti',
 'delinq_2yrs',
 'earliest_cr_line',
 'inq_last_6mths',
 'mths_since_last_delinq',
 'mths_since_last_record',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'revol_util',
 'total_acc',
 'initial_list_status',
 'out_prncp',
 'out_prncp_inv',
 'total_pymnt',
 'total_pymnt_inv',
 'total_rec_prncp',
 'total_rec_int',
 'total_rec_late_fee',
 'recoveries',
 'collection_recovery_fee',
 'last_pymnt_d',
 'last_pymnt_amnt',
 'next_pymnt_d',
 'last_credit_pull_d',
 'collections_12_mths_ex_med',
 'mths_since_last_major_derog',
 'policy_code',
 'not_compliant',
 'status',
 'inactive_loans',
 'bad_loans',
 'emp_length_num',
 'grade_num',
 'sub_grade_num',
 'delinq_2yrs_zero',
 'pub_rec

Here, we see that we have some feature columns that have to do with grade of the loan, annual income, home ownership status, etc. Let's take a look at the distribution of loan grades in the dataset.

Aquí, vemos que tenemos algunas columnas de características que tienen que ver con la calificación del préstamo, el ingreso anual, el estado de propiedad de la vivienda, etc. Echemos un vistazo a la distribución de las calificaciones de los préstamos en el conjunto de datos.

In [5]:
loans['grade'].show()

Materializing SArray

We can see that over half of the loan grades are assigned values `B` or `C`. Each loan is assigned one of these grades, along with a more finely discretized feature called `sub_grade` (feel free to explore that feature column as well!). These values depend on the loan application and credit report, and determine the interest rate of the loan. More information can be found [here](https://www.lendingclub.com/public/rates-and-fees.action).

Now, let's look at a different feature.

In [6]:
loans['home_ownership'].show()

Materializing SArray

This feature describes whether the loanee is mortaging, renting, or owns a home. We can see that a small percentage of the loanees own a home.

Esta característica describe si el prestatario está hipotecando, alquilando o es dueño de una casa. Podemos ver que un pequeño porcentaje de los prestatarios son propietarios de una vivienda.

## Exploring the target column

The target column (label column) of the dataset that we are interested in is called `bad_loans`. In this column **1** means a risky (bad) loan **0** means a safe  loan.

In order to make this more intuitive and consistent with the lectures, we reassign the target to be:
* **+1** as a safe  loan, 
* **-1** as a risky (bad) loan. 

We put this in a new column called `safe_loans`.

In [7]:
# safe_loans =  1 => safe
# safe_loans = -1 => risky
loans['safe_loans'] = loans['bad_loans'].apply(lambda x : +1 if x==0 else -1)
loans = loans.remove_column('bad_loans')

Now, let us explore the distribution of the column `safe_loans`. This gives us a sense of how many safe and risky loans are present in the dataset.

In [8]:
loans['safe_loans'].show()

Materializing SArray

You should have:
* Around 81% safe loans
* Around 19% risky loans

It looks like most of these loans are safe loans (thankfully). But this does make our problem of identifying risky loans challenging.

Parece que la mayoría de estos préstamos son préstamos seguros (afortunadamente). Pero esto hace que nuestro problema de identificar préstamos riesgosos sea un desafío.

## Features for the classification algorithm

In this assignment, we will be using a subset of features (categorical and numeric). The features we will be using are **described in the code comments** below. If you are a finance geek, the [LendingClub](https://www.lendingclub.com/) website has a lot more details about these features.

In [9]:
features = ['grade',                     # grade of the loan
            'sub_grade',                 # sub-grade of the loan
            'short_emp',                 # one year or less of employment
            'emp_length_num',            # number of years of employment
            'home_ownership',            # home_ownership status: own, mortgage or rent
            'dti',                       # debt to income ratio
            'purpose',                   # the purpose of the loan
            'term',                      # the term of the loan
            'last_delinq_none',          # has borrower had a delinquincy
            'last_major_derog_none',     # has borrower had 90 day or worse rating
            'revol_util',                # percent of available credit being used
            'total_rec_late_fee',        # total late fees received to day
           ]

target = 'safe_loans'                   # prediction target (y) (+1 means safe, -1 is risky)

# Extract the feature columns and target column
loans = loans[features + [target]]

In [10]:
loans

grade,sub_grade,short_emp,emp_length_num,home_ownership,dti,purpose,term,last_delinq_none
B,B2,0,11,RENT,27.65,credit_card,36 months,1
C,C4,1,1,RENT,1.0,car,60 months,1
C,C5,0,11,RENT,8.72,small_business,36 months,1
C,C1,0,11,RENT,20.0,other,36 months,0
A,A4,0,4,RENT,11.2,wedding,36 months,1
E,E1,0,10,RENT,5.35,car,36 months,1
F,F2,0,5,OWN,5.55,small_business,60 months,1
B,B5,1,1,RENT,18.08,other,60 months,1
C,C3,0,6,OWN,16.12,debt_consolidation,60 months,1
B,B5,0,11,OWN,10.78,debt_consolidation,36 months,1


What remains now is a **subset of features** and the **target** that we will use for the rest of this notebook. 

## Sample data to balance classes

As we explored above, our data is disproportionally full of safe loans.  Let's create two datasets: one with just the safe loans (`safe_loans_raw`) and one with just the risky loans (`risky_loans_raw`).

In [11]:
safe_loans_raw = loans[loans[target] == +1]
risky_loans_raw = loans[loans[target] == -1]
print("Number of safe loans  : %s" % len(safe_loans_raw))
print("Number of risky loans : %s" % len(risky_loans_raw))

Number of safe loans  : 99457
Number of risky loans : 23150


Now, write some code to compute below the percentage of safe and risky loans in the dataset and validate these numbers against what was given using `.show` earlier in the assignment:

In [13]:
print("Percentage of safe loans  :", 100 * len(safe_loans_raw) / float(len(loans))) 
print("Percentage of risky loans :", 100 * len(risky_loans_raw) / float(len(loans)))

Percentage of safe loans  : 81.11853319957262
Percentage of risky loans : 18.881466800427383


One way to combat class imbalance is to undersample the larger class until the class distribution is approximately half and half. Here, we will undersample the larger class (safe loans) in order to balance out our dataset. This means we are throwing away many data points. We used `seed=1` so everyone gets the same results.

Una forma de combatir el desequilibrio de clases es submuestrear la clase más grande hasta que la distribución de clases sea aproximadamente la mitad y la mitad. Aquí, submuestrearemos la clase más grande (préstamos seguros) para equilibrar nuestro conjunto de datos. Esto significa que estamos desechando muchos puntos de datos. Usamos seed=1 para que todos obtengan los mismos resultados.

In [14]:
# Since there are fewer risky loans than safe loans, find the ratio of the sizes
# and use that percentage to undersample the safe loans.
percentage = len(risky_loans_raw)/float(len(safe_loans_raw))

risky_loans = risky_loans_raw
safe_loans = safe_loans_raw.sample(percentage, seed=1)

# Append the risky_loans with the downsampled version of safe_loans
# Agregue o unir los préstamos_riesgosos con la versión reducida de préstamos_seguros
loans_data = risky_loans.append(safe_loans)

In [16]:
risky_loans

grade,sub_grade,short_emp,emp_length_num,home_ownership,dti,purpose,term,last_delinq_none
C,C4,1,1,RENT,1.0,car,60 months,1
F,F2,0,5,OWN,5.55,small_business,60 months,1
B,B5,1,1,RENT,18.08,other,60 months,1
C,C1,1,1,RENT,10.08,debt_consolidation,36 months,1
B,B2,0,4,RENT,7.06,other,36 months,1
B,B4,0,11,RENT,13.22,debt_consolidation,36 months,1
B,B3,0,2,RENT,2.4,major_purchase,36 months,1
C,C2,0,10,RENT,15.22,debt_consolidation,36 months,1
D,D2,0,3,RENT,13.97,other,60 months,0
A,A5,0,11,MORTGAGE,16.33,debt_consolidation,36 months,1


In [18]:
safe_loans

grade,sub_grade,short_emp,emp_length_num,home_ownership,dti,purpose,term,last_delinq_none
B,B3,0,11,OWN,11.18,credit_card,36 months,1
B,B1,0,3,MORTGAGE,29.44,credit_card,36 months,1
B,B3,1,1,RENT,12.19,credit_card,36 months,1
A,A4,0,6,MORTGAGE,14.03,debt_consolidation,36 months,1
C,C5,0,8,RENT,6.35,credit_card,36 months,1
B,B2,0,11,RENT,11.8,credit_card,36 months,1
B,B4,0,2,RENT,10.62,debt_consolidation,36 months,1
A,A2,0,6,RENT,10.85,debt_consolidation,36 months,1
B,B1,0,2,RENT,8.11,credit_card,36 months,1
B,B2,0,3,RENT,19.14,credit_card,36 months,1


In [19]:
loans_data

grade,sub_grade,short_emp,emp_length_num,home_ownership,dti,purpose,term,last_delinq_none
C,C4,1,1,RENT,1.0,car,60 months,1
F,F2,0,5,OWN,5.55,small_business,60 months,1
B,B5,1,1,RENT,18.08,other,60 months,1
C,C1,1,1,RENT,10.08,debt_consolidation,36 months,1
B,B2,0,4,RENT,7.06,other,36 months,1
B,B4,0,11,RENT,13.22,debt_consolidation,36 months,1
B,B3,0,2,RENT,2.4,major_purchase,36 months,1
C,C2,0,10,RENT,15.22,debt_consolidation,36 months,1
D,D2,0,3,RENT,13.97,other,60 months,0
A,A5,0,11,MORTGAGE,16.33,debt_consolidation,36 months,1


Now, let's verify that the resulting percentage of safe and risky loans are each nearly 50%.

In [20]:
print("Percentage of safe loans                 :", len(safe_loans) / float(len(loans_data)))
print("Percentage of risky loans                :", len(risky_loans) / float(len(loans_data)))
print("Total number of loans in our new dataset :", len(loans_data))

Percentage of safe loans                 : 0.5022361744216048
Percentage of risky loans                : 0.4977638255783951
Total number of loans in our new dataset : 46508


**Note:** There are many approaches for dealing with imbalanced data, including some where we modify the learning algorithm. These approaches are beyond the scope of this course, but some of them are reviewed in this [paper](http://ieeexplore.ieee.org/xpl/login.jsp?tp=&arnumber=5128907&url=http%3A%2F%2Fieeexplore.ieee.org%2Fiel5%2F69%2F5173046%2F05128907.pdf%3Farnumber%3D5128907 ). For this assignment, we use the simplest possible approach, where we subsample the overly represented class to get a more balanced dataset. In general, and especially when the data is highly imbalanced, we recommend using more advanced methods.

Nota: Existen muchos enfoques para manejar datos desequilibrados, incluidos algunos en los que modificamos el algoritmo de aprendizaje. Estos enfoques están más allá del alcance de este curso, pero algunos de ellos se revisan en este documento. Para esta tarea, usamos el enfoque más simple posible, donde submuestreamos la clase sobrerepresentada para obtener un conjunto de datos más equilibrado. En general, y especialmente cuando los datos están muy desequilibrados, recomendamos utilizar métodos más avanzados.

## Split data into training and validation sets

## El dataset de trabajo es ahora **Loans_data**

We split the data into training and validation sets using an 80/20 split and specifying `seed=1` so everyone gets the same results.

**Note**: In previous assignments, we have called this a **train-test split**. However, the portion of data that we don't train on will be used to help **select model parameters** (this is known as model selection). Thus, this portion of data should be called a **validation set**. Recall that examining performance of various potential models (i.e. models with different parameters) should be on validation set, while evaluation of the final selected model should always be on test data. Typically, we would also save a portion of the data (a real test set) to test our final model on or use cross-validation on the training set to select our final model. But for the learning purposes of this assignment, we won't do that.

Nota: En asignaciones anteriores, hemos llamado a esto un **train-test split**. Sin embargo, la parte de los datos en los que no entrenamos se usará para ayudar a **seleccionar los parámetros del modelo** (esto se conoce como selección del modelo). Por lo tanto, esta porción de datos debe llamarse **validation set**. Recuerde que examinar el **performace** de varios modelos potenciales (es decir, modelos con diferentes parámetros) debe estar en el **validation set**, mientras que la evaluación del modelo final seleccionado siempre debe estar en el **test data**. Por lo general, también guardaríamos una parte de los datos (un **test set** real) para probar nuestro modelo final o utilizar **cross-validation** en el **training set** para seleccionar nuestro modelo final. Pero para los propósitos de aprendizaje de esta tarea, no lo haremos.

In [21]:
train_data, validation_data = loans_data.random_split(.8, seed=1)

# Use decision tree to build a classifier

Now, let's use the built-in Turi Create decision tree learner to create a loan prediction model on the training data. (In the next assignment, you will implement your own decision tree learning algorithm.)  Our feature columns and target column have already been decided above. Use `validation_set=None` to get the same results as everyone else.

In [22]:
decision_tree_model = tc.decision_tree_classifier.create(train_data,
                                                                 validation_set=None,
                                                                 target = target,
                                                                 features = features)

Decision tree classifier:

--------------------------------------------------------

Number of examples          : 37224

Number of classes           : 2

Number of feature columns   : 12

...

+-----------+--------------+-------------------+-------------------+

| Iteration | Elapsed Time | Training Accuracy | Training Log Loss |

+-----------+--------------+-------------------+-------------------+

| 1         | 0.059467     | 0.640581          | 0.631396          |

+-----------+--------------+-------------------+-------------------+

## Building a smaller tree

Typically the max depth of the tree is capped at 6. However, such a tree can be hard to visualize graphically, and moreover, it may overfit..  Here, we instead learn a smaller model with **max depth of 2** to gain some intuition and to understand the learned tree more.

Por lo general, la profundidad máxima del árbol tiene un límite de 6. Sin embargo, tal árbol puede ser difícil de visualizar gráficamente y, además, puede sobreajustarse. Aquí, en cambio, aprendemos un modelo más pequeño con una **profundidad máxima de 2** para ganar algo de intuición y para entender más el árbol aprendido.

In [49]:
small_model = tc.decision_tree_classifier.create(train_data,
                                                         validation_set=None,
                                                         target = target,
                                                         features = features,
                                                         max_depth = 2)

Decision tree classifier:

--------------------------------------------------------

Number of examples          : 37224

Number of classes           : 2

Number of feature columns   : 12

...

+-----------+--------------+-------------------+-------------------+

| Iteration | Elapsed Time | Training Accuracy | Training Log Loss |

+-----------+--------------+-------------------+-------------------+

| 1         | 0.048838     | 0.613502          | 0.658759          |

+-----------+--------------+-------------------+-------------------+

# Making predictions

Let's consider two positive and two negative examples **from the validation set** and see what the model predicts. We will do the following:
* Predict whether or not a loan is safe.
* Predict the probability that a loan is safe.

Consideremos dos ejemplos positivos y dos negativos **from the validation set** y veamos qué predice el modelo. Haremos lo siguiente:

* Predecir si un préstamo es seguro o no.
* Predecir la probabilidad de que un préstamo sea seguro.

In [24]:
validation_safe_loans = validation_data[validation_data[target] == 1]
validation_risky_loans = validation_data[validation_data[target] == -1]

sample_validation_data_risky = validation_risky_loans[0:2]
sample_validation_data_safe = validation_safe_loans[0:2]

sample_validation_data = sample_validation_data_safe.append(sample_validation_data_risky)
sample_validation_data

Columns:
	grade	str
	sub_grade	str
	short_emp	int
	emp_length_num	int
	home_ownership	str
	dti	float
	purpose	str
	term	str
	last_delinq_none	int
	last_major_derog_none	int
	revol_util	float
	total_rec_late_fee	float
	safe_loans	int

Rows: 4

Data:
+-------+-----------+-----------+----------------+----------------+-------+
| grade | sub_grade | short_emp | emp_length_num | home_ownership |  dti  |
+-------+-----------+-----------+----------------+----------------+-------+
|   B   |     B3    |     0     |       11       |      OWN       | 11.18 |
|   D   |     D1    |     0     |       10       |      RENT      | 16.85 |
|   D   |     D2    |     0     |       3        |      RENT      | 13.97 |
|   A   |     A5    |     0     |       11       |    MORTGAGE    | 16.33 |
+-------+-----------+-----------+----------------+----------------+-------+
+--------------------+------------+------------------+-----------------------+
|      purpose       |    term    | last_delinq_none | last_major_derog_none |
+--------------------+------------+------------------+-----------------------+
|    credit_card     |  36 months |        1         |           1           |
| debt_consolidation |  36 months |        1         |           1           |
|       other        |  60 months |        0         |           1           |
| debt_consolidation |  36 months |        1         |           1           |
+--------------------+------------+------------------+-----------------------+
+------------+--------------------+------------+
| revol_util | total_rec_late_fee | safe_loans |
+------------+--------------------+------------+
|    82.4    |        0.0         |     1      |
|    96.4    |        0.0         |     1      |
|    59.5    |        0.0         |     -1     |
|    62.1    |        0.0         |     -1     |
+------------+--------------------+------------+
[4 rows x 13 columns]

## Explore label predictions

Now, we will use our model  to predict whether or not a loan is likely to default. For each row in the **sample_validation_data**, use the **decision_tree_model** to predict whether or not the loan is classified as a **safe loan**. 

**Hint:** Be sure to use the `.predict()` method.

In [27]:
predictions = decision_tree_model.predict(sample_validation_data, output_type='class')

In [29]:
predictions

dtype: int
Rows: 4
[1, -1, -1, 1]

In [37]:
# Comparo cuantos elementos son parecidos y cuento los TRUE

compara = (sample_validation_data['safe_loans'] == decision_tree_model.predict(sample_validation_data)).sum()

porcent = compara/float(len(sample_validation_data))

porcent

0.5

**Quiz Question:** What percentage of the predictions on `sample_validation_data` did `decision_tree_model` get correct?

**R/:** 50%

## Explore probability predictions

For each row in the **sample_validation_data**, what is the probability (according **decision_tree_model**) of a loan being classified as **safe**? 


**Hint:** Set `output_type='probability'` to make **probability** predictions using **decision_tree_model** on `sample_validation_data`:

In [38]:
probability_predictions = decision_tree_model.predict(sample_validation_data, output_type='probability')
probability_predictions

dtype: float
Rows: 4
[0.6532223224639893, 0.463798463344574, 0.356814444065094, 0.7621196508407593]

**Quiz Question:** Which loan has the highest probability of being classified as a **safe loan**?

**Checkpoint:** Can you verify that for all the predictions with `probability >= 0.5`, the model predicted the label **+1**?

In [46]:
sample_validation_data['predictions'] = predictions
sample_validation_data['probability'] = probability_predictions

sample_validation_data['safe_loans','predictions','probability']

safe_loans,predictions,probability
1,1,0.6532223224639893
1,-1,0.463798463344574
-1,-1,0.356814444065094
-1,1,0.7621196508407593


### Tricky predictions!

Now, we will explore something pretty interesting. For each row in the **sample_validation_data**, what is the probability (according to **small_model**) of a loan being classified as **safe**?

**Hint:** Set `output_type='probability'` to make **probability** predictions using **small_model** on `sample_validation_data`:

In [47]:
probability_predictions_small = small_model.predict(sample_validation_data, output_type='probability')
probability_predictions_small

dtype: float
Rows: 4
[0.5803016424179077, 0.4085058867931366, 0.4085058867931366, 0.7454202175140381]

**Quiz Question:** Notice that the probability preditions are the **exact same** for the 2nd and 3rd loans. Why would this happen?

**R/:** 
* During tree traversal both examples fall into the same leaf node.
* Durante el recorrido del árbol, ambos ejemplos caen en el mismo nodo hoja.

In [48]:
sample_validation_data['probability small model'] = probability_predictions_small

sample_validation_data['safe_loans','predictions','probability','probability small model']

safe_loans,predictions,probability,probability small model
1,1,0.6532223224639893,0.5803016424179077
1,-1,0.463798463344574,0.4085058867931366
-1,-1,0.356814444065094,0.4085058867931366
-1,1,0.7621196508407593,0.7454202175140381


# Evaluating accuracy of the decision tree model

Recall that the accuracy is defined as follows:
$$
\mbox{accuracy} = \frac{\mbox{# correctly classified examples}}{\mbox{# total examples}}
$$

Let us start by evaluating the accuracy of the `small_model` and `decision_tree_model` on the training data

In [50]:
print(small_model.evaluate(train_data)['accuracy'])
print(decision_tree_model.evaluate(train_data)['accuracy'])

0.6135020416935311
0.6405813453685794


**Checkpoint:** You should see that the **small_model** performs worse than the **decision_tree_model** on the training data.


Now, let us evaluate the accuracy of the **small_model** and **decision_tree_model** on the entire **validation_data**, not just the subsample considered above.

**Checkpoint:**: debería ver que el **small_model** funciona peor que el **decision_tree_model** en los datos de entrenamiento.

Ahora, permítanos evaluar la precisión de **small_model** y **decision_tree_model** en todos el **validation_data**, no solo en la submuestra considerada anteriormente.

In [51]:
print (small_model.evaluate(validation_data)['accuracy'])
print (decision_tree_model.evaluate(validation_data)['accuracy'])

0.6193451098664369
0.6367944851357173


**Quiz Question:** What is the accuracy of `decision_tree_model` on the validation set, rounded to the nearest .01?

**R/:** 0.64

## Evaluating accuracy of a complex decision tree model

Here, we will train a large decision tree with `max_depth=10`. This will allow the learned tree to become very deep, and result in a very complex model. Recall that in lecture, we prefer simpler models with similar predictive power. This will be an example of a more complicated model which has similar predictive power, i.e. something we don't want.

In [52]:
big_model = tc.decision_tree_classifier.create(train_data, validation_set=None,
                                                       target = target, features = features, max_depth = 10)

Decision tree classifier:

--------------------------------------------------------

Number of examples          : 37224

Number of classes           : 2

Number of feature columns   : 12

...

+-----------+--------------+-------------------+-------------------+

| Iteration | Elapsed Time | Training Accuracy | Training Log Loss |

+-----------+--------------+-------------------+-------------------+

| 1         | 0.092019     | 0.665538          | 0.606828          |

+-----------+--------------+-------------------+-------------------+

Now, let us evaluate **big_model** on the training set and validation set.

In [53]:
print(big_model.evaluate(train_data)['accuracy'])
print(big_model.evaluate(validation_data)['accuracy'])

0.665538362346873
0.6274235243429557


**Checkpoint:** We should see that **big_model** has even better performance on the training set than **decision_tree_model** did on the training set.

In [62]:
print(big_model.evaluate(train_data)['accuracy'])
print(decision_tree_model.evaluate(train_data)['accuracy'])

0.665538362346873
0.6405813453685794


**Quiz Question:** How does the performance of **big_model** on the validation set compare to **decision_tree_model** on the validation set? Is this a sign of overfitting?

**R/:** Lower. big_model has lower accuracy on the validation set than decision_tree_model. This is overfitting.

In [64]:
print(big_model.evaluate(validation_data)['accuracy'])
print(decision_tree_model.evaluate(validation_data)['accuracy'])

0.6274235243429557
0.6367944851357173


### Quantifying the cost of mistakes

Every mistake the model makes costs money. In this section, we will try and quantify the cost of each mistake made by the model.

Assume the following:

* **False negatives**: Loans that were actually safe but were predicted to be risky. This results in an oppurtunity cost of losing a loan that would have otherwise been accepted. 
* **False positives**: Loans that were actually risky but were predicted to be safe. These are much more expensive because it results in a risky loan being given. 
* **Correct predictions**: All correct predictions don't typically incur any cost.


Let's write code that can compute the cost of mistakes made by the model. Complete the following 4 steps:
1. First, let us compute the predictions made by the model.
1. Second, compute the number of false positives.
2. Third, compute the number of false negatives.
3. Finally, compute the cost of mistakes made by the model by adding up the costs of true positives and false positives.

First, let us make predictions on `validation_data` using the `decision_tree_model`:

## Cuantificar el costo de los errores

Cada error que comete el modelo cuesta dinero. En este apartado intentaremos cuantificar el coste de cada error cometido por el modelo.

Suponga lo siguiente:

* **Falsos negativos**: Préstamos que en realidad eran seguros pero que se predijo que serían riesgosos. Esto da como resultado un costo de oportunidad de perder un préstamo que de otro modo habría sido aceptado.
* **Falsos positivos**: Préstamos que en realidad eran riesgosos pero que se predijo que serían seguros. Estos son mucho más caros porque dan como resultado un préstamo riesgoso.
* **Predicciones correctas**: todas las predicciones correctas no suelen generar ningún costo.

Escribamos un código que pueda calcular el costo de los errores cometidos por el modelo. Complete los siguientes 4 pasos:

1. Primero, calculemos las predicciones hechas por el modelo.
2. En segundo lugar, calcule el número de falsos positivos.
3. Tercero, calcule el número de falsos negativos.
4. Finalmente, calcule el costo de los errores cometidos por el modelo sumando los costos de los verdaderos positivos y los falsos positivos.

Primero, hagamos predicciones sobre `validation_data` usando el `decision_tree_model`:

In [54]:
predictions = decision_tree_model.predict(validation_data)

**False positives** are predictions where the model predicts +1 but the true label is -1. Complete the following code block for the number of false positives:

In [55]:
validation_data['predictions'] = predictions
validation_data['safe_loans','predictions']

safe_loans,predictions
-1,-1
-1,1
-1,-1
-1,-1
-1,1
-1,-1
-1,1
-1,1
-1,-1
-1,-1


In [65]:
false_positives = (validation_data[validation_data['safe_loans'] != predictions]['safe_loans'] == -1).sum()
print (false_positives)

1656


In [68]:
false_positives = sum((predictions ==1)&(validation_data['safe_loans']==-1))
print (false_positives)

1656


**False negatives** are predictions where the model predicts -1 but the true label is +1. Complete the following code block for the number of false negatives:

In [69]:
false_negatives = (validation_data[validation_data['safe_loans'] != predictions]['safe_loans'] == +1).sum()
print (false_negatives)

1716


In [70]:
false_negatives = sum((predictions ==-1)&(validation_data['safe_loans']==1))
print (false_negatives)

1716


**Quiz Question:** Let us assume that each mistake costs money:
* Assume a cost of \$10,000 per false negative.
* Assume a cost of \$20,000 per false positive.

What is the total cost of mistakes made by `decision_tree_model` on `validation_data`?

In [60]:
cost_false_negatives = 10000 * false_negatives
cost_false_positives = 20000 * false_positives

print (cost_false_negatives + cost_false_positives)

50280000
